In [1]:
import pandas as pd
import numpy as np
import arrow
import os

In [2]:
TRAIN_DATA_PATH = os.path.join("data","train.csv")
TEST_DATA_PATH = os.path.join("data", "test.csv")
METADATA_FILEPATH = os.path.join("data","item_metadata.csv")
SAMPLED_DATA_PATH = os.path.join("data", "sampled", "train_sample.csv")
EVENTS_WRITE_PATH = os.path.join("data", "events.csv")

In [3]:
def get_dataframe_memory_usage(df):
    return df.memory_usage(deep=True).sum() * 10**(-9)

In [ ]:
train_data = pd.read_csv(TRAIN_DATA_PATH)
test_data = pd.read_csv(TEST_DATA_PATH)

In [6]:
train_data.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [7]:
test_data.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,004A07DM0IDW,1d688ec168932,1541555614,1,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
1,004A07DM0IDW,1d688ec168932,1541555614,2,interaction item image,2059240,CO,"Santa Marta, Colombia",mobile,NaN,NaN,NaN
2,004A07DM0IDW,1d688ec168932,1541555696,3,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
3,004A07DM0IDW,1d688ec168932,1541555707,4,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
4,004A07DM0IDW,1d688ec168932,1541555717,5,clickout item,1050068,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...


In [8]:
np.all(train_data.columns == test_data.columns)

True

In [9]:
# continue with subset because of memory limitations
del train_data
del test_data

In [4]:
events = pd.read_csv(SAMPLED_DATA_PATH, index_col=0)

In [5]:
### DATA PREPARATION ###
# This part is highly inspired from the Logic AI, Layer6 AI and PVZ teams' solutions to the challenge.

events["src"] = "train"
events["is_test"] = 0

events.sort_values(["timestamp", "user_id", "step"], inplace=True)

# fill empty impressions with backward filling, impressions carried to earlier steps
events["fake_impressions"] = events.groupby(["user_id", "session_id"])["impressions"].bfill()
events["fake_prices"] = events.groupby(["user_id", "session_id"])["prices"].bfill()

events["reversed_clickout_step"] = (
    events.groupby(["action_type", "session_id"])["step"].rank("max", ascending=False).astype(int)
)

events["clickout_step"] = (
    events.groupby(["action_type", "session_id"])["step"].rank("max", ascending=True).astype(int)
)

events["clickout_max_step"] = events["clickout_step"] + events["reversed_clickout_step"] - 1

events["dt"] = events["timestamp"].apply(lambda x: str(arrow.get(x).date()))

In [6]:
events.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,src,is_test,fake_impressions,fake_prices,reversed_clickout_step,clickout_step,clickout_max_step,dt
578181,RE84XPQB447X,90fdaae2c58dc,1541030423,1,search for poi,Disneyland Paris,US,"Marne-la-Vallée, France",desktop,Best Value,NaN,NaN,train,0,8887828|8414618|1231946|7323680|5125642|415471...,98|119|105|98|145|120|156|145|180|125|168|138|...,2,1,2,2018-11-01
97159,WDPJ442S0Q1Z,cf0c96d88d2ef,1541030432,1,search for destination,"Birmingham, United Kingdom",UK,"Birmingham, United Kingdom",mobile,NaN,NaN,NaN,train,0,449896|147730|12522|152022|12544|12531|12516|2...,117|127|117|101|125|70|124|95|115|73|127|123|1...,3,1,3,2018-11-01
735508,PCYB9NJCV2IS,47ba72122b04c,1541030442,1,search for destination,"Lisbon, Portugal",US,"Lisbon, Portugal",mobile,NaN,NaN,NaN,train,0,NaN,NaN,1,1,1,2018-11-01
578182,RE84XPQB447X,90fdaae2c58dc,1541030454,2,interaction item rating,149255,US,"Marne-la-Vallée, France",desktop,NaN,NaN,NaN,train,0,8887828|8414618|1231946|7323680|5125642|415471...,98|119|105|98|145|120|156|145|180|125|168|138|...,4,1,4,2018-11-01
421874,ORQ6U0KAMD2Q,73aaba45abc79,1541030468,1,interaction item rating,69865,US,"Red Lodge, USA",desktop,NaN,NaN,NaN,train,0,69511|69865|3867636|8258580|2217944|824256|893...,55|89|61|70|84|141|106|212|75|123|92,5,1,5,2018-11-01


In [7]:
# events that occur at 2018 - 11 - 06 
num_of_validation_clickouts = events[(events["dt"] == "2018-11-06") & (events["action_type"] == "clickout item")].shape[0]

num_of_clickouts = events[events["action_type"] == "clickout item"].shape[0]

print(f"Validation ratio {round(num_of_validation_clickouts/num_of_clickouts, 2)}")

Validation ratio 0.17


In [8]:
get_dataframe_memory_usage(events)

0.932077103

In [9]:
validation_events = events.loc[(events["dt"] == "2018-11-06") & (events["action_type"] == "clickout item")].copy(deep=True)

validation_events["reversed_user_clickout_step"] = (
    validation_events.groupby(["action_type", "user_id"])["step"].rank("max", ascending=False).astype(int)
)

last_clickouts_in_validation = validation_events[validation_events["reversed_user_clickout_step"] == 1][["user_id", "session_id", "step"]]
last_clickouts_in_validation["is_val"] = 1

last_clickouts_in_validation.head()

events = pd.merge(events, last_clickouts_in_validation, on=["user_id", "session_id", "step"], how="left")


events["is_val"].fillna(0, inplace=True)
events["is_val"] = events["is_val"].astype(int)

events.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,...,prices,src,is_test,fake_impressions,fake_prices,reversed_clickout_step,clickout_step,clickout_max_step,dt,is_val
0,RE84XPQB447X,90fdaae2c58dc,1541030423,1,search for poi,Disneyland Paris,US,"Marne-la-Vallée, France",desktop,Best Value,...,NaN,train,0,8887828|8414618|1231946|7323680|5125642|415471...,98|119|105|98|145|120|156|145|180|125|168|138|...,2,1,2,2018-11-01,0
1,WDPJ442S0Q1Z,cf0c96d88d2ef,1541030432,1,search for destination,"Birmingham, United Kingdom",UK,"Birmingham, United Kingdom",mobile,NaN,...,NaN,train,0,449896|147730|12522|152022|12544|12531|12516|2...,117|127|117|101|125|70|124|95|115|73|127|123|1...,3,1,3,2018-11-01,0
2,PCYB9NJCV2IS,47ba72122b04c,1541030442,1,search for destination,"Lisbon, Portugal",US,"Lisbon, Portugal",mobile,NaN,...,NaN,train,0,NaN,NaN,1,1,1,2018-11-01,0
3,RE84XPQB447X,90fdaae2c58dc,1541030454,2,interaction item rating,149255,US,"Marne-la-Vallée, France",desktop,NaN,...,NaN,train,0,8887828|8414618|1231946|7323680|5125642|415471...,98|119|105|98|145|120|156|145|180|125|168|138|...,4,1,4,2018-11-01,0
4,ORQ6U0KAMD2Q,73aaba45abc79,1541030468,1,interaction item rating,69865,US,"Red Lodge, USA",desktop,NaN,...,NaN,train,0,69511|69865|3867636|8258580|2217944|824256|893...,55|89|61|70|84|141|106|212|75|123|92,5,1,5,2018-11-01,0


In [10]:
events.isna().sum()

user_id                        0
session_id                     0
timestamp                      0
step                           0
action_type                    0
reference                      0
platform                       0
city                           0
device                         0
current_filters           815220
impressions               792293
prices                    792293
src                            0
is_test                        0
fake_impressions          255995
fake_prices               255995
reversed_clickout_step         0
clickout_step                  0
clickout_max_step              0
dt                             0
is_val                         0
dtype: int64

In [11]:
events.to_csv(EVENTS_WRITE_PATH)